<a href="https://colab.research.google.com/github/Baran-7/ari_ring/blob/main/airring_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

H
the code to optimize airrinig and extracting the formula  
230 total data

In [ ]:
b

In [ ]:
import pandas as pd
import torch
from torch import nn
!pip install openpyxl
import torch.optim as optim
import numpy as np
import torch.optim as optim

loading the data from the excel file using pandas



In [ ]:
import pandas as pd
import torch
#connecting to the one drive and I choosed agree on all the levels
from google.colab import drive
drive.mount('/content/drive')
#importing the exact folder name
file_path = "/content/drive/My Drive/air_ring/export airring data.xlsx"
#checking what is in the directory
#import os
# Adjust the path to your folder
#folder_path = "/content/drive/My Drive/air_ring"
#print(os.listdir(folder_path))
#loading the data
#data = pd.read_excel('/content/export airring data.xlsx'): the chat gpt's mistake :)))))
data = pd.read_excel('/content/drive/My Drive/air_ring/export airring data.xlsx')
ten_raw_data = torch.tensor(data.values, dtype = torch.float32)

#extacting the data from the hole and spiliting

#train set
num_train_eg = 231
train_sec = 2/3
train_data = torch.transpose(ten_raw_data[1:int(num_train_eg*train_sec),1:7],0,1)
#test set
test_data = torch.transpose(ten_raw_data[int(num_train_eg*train_sec):num_train_eg,1:8],0,1)
#output train set
output_train_data = torch.transpose(ten_raw_data[1:int(num_train_eg*train_sec), 8].unsqueeze(1), 1, 0)
#output test set
output_test_data = torch.transpose(ten_raw_data[1:int(num_train_eg*train_sec), 8].unsqueeze(1), 1, 0)


Mounted at /content/drive


In [ ]:
#data visualization and checking
#print(train_data)
print(train_data.shape)
print(output_train_data.shape)


torch.Size([6, 153])
torch.Size([1, 153])


we will start with a two layer neural network in which the second one is the output layer

In [ ]:
# Make device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
#nonlinear regression

from torch import nn
class nonlinearregression(nn.Module):
  def __init__(self):
    super().__init__()
    #the model architecture
    # lets creat a 1 hidden layer neural network but infdeed 3 layers
    self.layer_1 = nn.Linear(in_features=6, out_features=10)
    self.layer_2 = nn.Linear(in_features=10, out_features=1)

  def forward(self, train_data):
    return(self.layer_2(self.layer_1(train_data)))

model_0 = nonlinearregression().to(device)
model_0

nonlinearregression(
  (layer_1): Linear(in_features=6, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=1, bias=True)
)

In [ ]:
untrained_preds = model_0(test_data.to(device))
print(f"Length of predictions: {len(untrained_preds)}, Shape: {untrained_preds.shape}")
print(f"Length of test samples: {len(output_test_data)}, Shape: {output_test_data.shape}")
print(f"\nFirst 10 predictions:\n{untrained_preds[:10]}")
print(f"\nFirst 10 test labels:\n{output_test_data[:10]}")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (7x76 and 6x10)